In [1]:
import pandas as pd

df = pd.read_pickle('df.pkl')

agg_functions = {
    'gene_id': 'first',
    'combined nucleotides': 'first',
    'dwelling_time1': 'mean',
    'sd1': 'mean',
    'mean1': 'mean',
    'dwelling_time2': 'mean',
    'sd2': 'mean',
    'mean2': 'mean',
    'dwelling_time3': 'mean',
    'sd3': 'mean',
    'mean3': 'mean',
    'label': 'sum'
}

summary_df = df.groupby(['transcript_id', 'transcript_position']).agg(agg_functions).reset_index()
summary_df['count'] = df.groupby(['transcript_id', 'transcript_position']).size().reset_index(name='count')['count']
summary_df['label_percentage'] = summary_df['label'] / summary_df['count']
summary_df = summary_df.rename(columns={'label': 'label_sum'})
summary_df['label'] = summary_df['label_percentage'].apply(lambda x: 0 if x == 0 else 1)
summary_df.drop('label_percentage', axis=1, inplace=True)
summary_df.drop('label_sum', axis=1, inplace=True)

### Tokenization of sequences and is_DRACH feature

In [2]:
# function to convert features
def sequence_to_features(seq):
    """
    Convert a 7-letter sequence into 28 binary features.
    
    Parameters:
    - seq: A string of 7 letters (A, T, C, G)
    
    Returns:
    - A dictionary with 28 binary features.
    """
    letters = ['A', 'T', 'C', 'G']
    features = {}
    
    for i, char in enumerate(seq, start=1):
        for letter in letters:
            feature_name = f"{i}{letter}"
            features[feature_name] = char == letter
            
    return features

In [3]:
new_df = summary_df.copy()

feature_dicts = new_df['combined nucleotides'].apply(sequence_to_features)
features_df = feature_dicts.apply(pd.Series)
new_df = pd.concat([new_df, features_df], axis=1)

In [4]:
def is_drach_motif(sequence):
    # Extract middle 5 nucleotides
    motif = sequence[1:6]
    
    # Check the DRACH rules
    D = motif[0] in ['A', 'G', 'T']
    R = motif[1] in ['A', 'G']
    A = motif[2] == 'A'
    C = motif[3] == 'C'
    H = motif[4] in ['A', 'C', 'T']

    return D and R and A and C and H

In [5]:
# Create a new column indicating whether the sequence satisfies the DRACH motif
new_df['is_DRACH'] = new_df['combined nucleotides'].apply(is_drach_motif)

new_df.drop('combined nucleotides', axis='columns', inplace=True)
new_df

,transcript_id,transcript_position,gene_id,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,...,5G,6A,6T,6C,6G,7A,7T,7C,7G,is_DRACH
0,ENST00000000233,244,ENSG00000004059,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,...,False,False,False,True,False,True,False,False,False,True
1,ENST00000000233,261,ENSG00000004059,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,...,False,False,True,False,False,False,False,False,True,True
2,ENST00000000233,316,ENSG00000004059,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,...,False,True,False,False,False,False,False,False,True,True
3,ENST00000000233,332,ENSG00000004059,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006101,...,False,True,False,False,False,False,True,False,False,True
4,ENST00000000233,368,ENSG00000004059,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,...,False,True,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,ENSG00000167747,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,0.006555,...,False,True,False,False,False,False,True,False,False,True
121834,ENST00000641834,1429,ENSG00000167747,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,0.005568,...,False,True,False,False,False,False,False,True,False,True
121835,ENST00000641834,1531,ENSG00000167747,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,0.006410,...,False,True,False,False,False,False,False,True,False,True
121836,ENST00000641834,1537,ENSG00000167747,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,0.006472,...,False,False,False,True,False,True,False,False,False,True


### Label Encoder

In [6]:
from sklearn.preprocessing import LabelEncoder

combined = new_df.copy()

label_encoder = LabelEncoder()
combined['transcript_id'] = label_encoder.fit_transform(combined['transcript_id'])
combined['gene_id'] = label_encoder.fit_transform(combined['gene_id'])
combined

,transcript_id,transcript_position,gene_id,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,...,5G,6A,6T,6C,6G,7A,7T,7C,7G,is_DRACH
0,0,244,10,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,...,False,False,False,True,False,True,False,False,False,True
1,0,261,10,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,...,False,False,True,False,False,False,False,False,True,True
2,0,316,10,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,...,False,True,False,False,False,False,False,False,True,True
3,0,332,10,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006101,...,False,True,False,False,False,False,True,False,False,True
4,0,368,10,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,...,False,True,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,5332,1348,2764,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,0.006555,...,False,True,False,False,False,False,True,False,False,True
121834,5332,1429,2764,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,0.005568,...,False,True,False,False,False,False,False,True,False,True
121835,5332,1531,2764,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,0.006410,...,False,True,False,False,False,False,False,True,False,True
121836,5332,1537,2764,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,0.006472,...,False,False,False,True,False,True,False,False,False,True


### Stratified K Fold

In [7]:
X = combined.drop('label', axis=1)
y = combined['label']

In [8]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks, RandomUnderSampler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_curve

# Initialize StratifiedKFold
strat_kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

# ADASYN followed by Tomek Links Pipeline
adasyn_tomek = Pipeline([
    ('adasyn', ADASYN(random_state=42)),
    ('tomek', TomekLinks())
])

# Dictionary of resampling methods
resampling_methods = {
    'RandomUnderSampler': RandomUnderSampler(random_state=42),
    'TomekLinks': TomekLinks(),
    'SMOTE': SMOTE(random_state=42),
    'ADASYN': ADASYN(random_state=42),
    'SMOTETomek': SMOTETomek(random_state=42),
    'ADASYN_Tomek': adasyn_tomek
}

# Model initialization
xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Loop through each resampling method
for method_name, resampler in resampling_methods.items():
    print(f"Results for {method_name}:")
    
    roc_aucs = []  # List to store ROC AUC scores for each fold
    
    # Loop through each fold
    for train_index, valid_index in strat_kfold.split(X, y):
        X_train_fold, X_valid_fold = X.iloc[train_index], X.iloc[valid_index]
        y_train_fold, y_valid_fold = y.iloc[train_index], y.iloc[valid_index]
        
        # Initialize the scaler
        scaler = MinMaxScaler()
        
        # Fit the scaler on the training data and transform both training and validation data
        X_train_fold_scaled = scaler.fit_transform(X_train_fold)
        X_valid_fold_scaled = scaler.transform(X_valid_fold)
        
        # Apply the current resampling technique
        X_resampled, y_resampled = resampler.fit_resample(X_train_fold_scaled, y_train_fold)
        
        # Train the model on the resampled data
        xgb_model.fit(X_resampled, y_resampled)
        
        # Predict on the validation set
        y_pred = xgb_model.predict_proba(X_valid_fold_scaled)[:, 1]
        
        threshold = 0.4
        y_pred_binary = (y_pred >= threshold).astype(int)
        
        # Compute the ROC AUC score for this fold and append to the list
        roc_aucs.append(roc_auc_score(y_valid_fold, y_pred))
    
    # Display results for the current resampling method
    print("ROC AUC scores for each fold:", roc_aucs)
    print("Mean ROC AUC score:", np.mean(roc_aucs))
    print("Standard deviation of ROC AUC scores:", np.std(roc_aucs))
    print("-" * 50)


Results for RandomUnderSampler:
ROC AUC scores for each fold: [0.914578223496489, 0.9071812889179007, 0.9138826872694503, 0.9038964848501972, 0.906915398754613]
Mean ROC AUC score: 0.90929081665773
Standard deviation of ROC AUC scores: 0.004200797065859132
--------------------------------------------------
Results for TomekLinks:
ROC AUC scores for each fold: [0.9206180678140954, 0.9176005000797561, 0.9248375888574508, 0.9159948027770847, 0.9170278116567856]
Mean ROC AUC score: 0.9192157542370346
Standard deviation of ROC AUC scores: 0.0032044398856007593
--------------------------------------------------
Results for SMOTE:
ROC AUC scores for each fold: [0.9036159839522429, 0.8989120793158513, 0.9115040907144049, 0.8979946897598541, 0.8976805764192689]
Mean ROC AUC score: 0.9019414840323243
Standard deviation of ROC AUC scores: 0.005237503378335239
--------------------------------------------------
Results for ADASYN:
ROC AUC scores for each fold: [0.9099093213037941, 0.894161910238744